In [ ]:
!pip install sklearn-crfsuite

Import Libraries


In [ ]:

import spacy
from spacy.tokens import Doc
nlp = spacy.load("en_core_web_sm")
import sklearn_crfsuite
from sklearn_crfsuite import metrics

import nltk
from nltk.corpus import brown
nltk.download('brown')
nltk.download('universal_tagset')

[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Package brown is already up-to-date!
[nltk_data] Downloading package universal_tagset to /root/nltk_data...
[nltk_data]   Package universal_tagset is already up-to-date!


True

In [ ]:
## Creating A function give me a list of lists

def token2features(my_sen):
    my_doc = Doc(nlp.vocab, words=my_sen)
    my_list = []

    for i  in range(len(my_doc)):
      features = {
          'lower': my_doc[i].lower_,
          'suffix': my_doc[i].suffix_,
          'prefix': my_doc[i].prefix_,
          'isupper': my_doc[i].is_upper,
          'istitle': my_doc[i].is_title,
          'isdigit': my_doc[i].is_digit
          }
      if i > 0:
          features.update({
              '-1_lower': my_doc[i-1].lower_,
              '-1_suffix': my_doc[i-1].suffix_,
              '-1_prefix': my_doc[i-1].prefix_,
              '-1_isupper': my_doc[i-1].is_upper,
              '-1_istitle': my_doc[i-1].is_title,
              '-1_isdigit': my_doc[i-1].is_digit
          })
      else:
          features['BOS'] = True

      if i < len(my_doc)-1:
          features.update({
              '+1_lower': my_doc[i+1].lower_,
              '+1_suffix': my_doc[i+1].suffix_,
              '+1_prefix': my_doc[i+1].prefix_,
              '+1_isupper': my_doc[i+1].is_upper,
              '+1_istitle': my_doc[i+1].is_title,
              '+1_isdigit': my_doc[i+1].is_digit,
          })
      else:
          features['EOS'] = True

      my_list.append(features)

    return my_list

In [ ]:
## Download the brown corpus from NLTK
my_sents = brown.tagged_sents(tagset="universal")
my_sents1 = brown.sents()

In [ ]:
## Calling token2features function to the first sentens in brown curpos

In [ ]:
inputs = [token2features(sent) for sent in my_sents1]
outputs = [[s[1] for s in lis] for lis in my_sents]

In [ ]:
## splitting the data
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(inputs,outputs, test_size=0.3,shuffle=False)

In [ ]:
## Building and training a POS tagger with CRF
crf = sklearn_crfsuite.CRF(
    algorithm='lbfgs',
    c1=0.1,
    c2=0.1,
    max_iterations=100,
    all_possible_transitions=True
)
crf.fit(X_train, y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/base.py:197: FutureWarning: From version 0.24, get_params will raise an AttributeError if a parameter cannot be retrieved as an instance attribute. Previously it would return None.
  FutureWarning)


CRF(algorithm='lbfgs', all_possible_states=None, all_possible_transitions=True,
    averaging=None, c=None, c1=0.1, c2=0.1, calibration_candidates=None,
    calibration_eta=None, calibration_max_trials=None, calibration_rate=None,
    calibration_samples=None, delta=None, epsilon=None, error_sensitive=None,
    gamma=None, keep_tempfiles=None, linesearch=None, max_iterations=100,
    max_linesearch=None, min_freq=None, model_filename=None, num_memories=None,
    pa_type=None, period=None, trainer_cls=None, variance=None, verbose=False)

In [ ]:
## output
labels = list(crf.classes_)

labels

['DET',
 'NOUN',
 'ADJ',
 'VERB',
 'ADP',
 '.',
 'ADV',
 'CONJ',
 'PRT',
 'PRON',
 'NUM',
 'X']

In [ ]:
## prediction
y_pred = crf.predict(X_test)
metrics.flat_f1_score(y_test, y_pred, average='weighted', labels=labels)

0.9715029906977743

In [ ]:
## checking accuracy
sorted_labels = sorted(labels,key=lambda name: (name[1:], name[0]))

print(metrics.flat_classification_report(
    y_test, y_pred, labels=sorted_labels, digits=3
))

              precision    recall  f1-score   support

           .      1.000     1.000     1.000     42531
           X      0.755     0.282     0.410       252
         ADJ      0.904     0.894     0.899     14333
         ADP      0.963     0.975     0.969     27622
         ADV      0.934     0.918     0.926     15343
        VERB      0.981     0.980     0.980     48403
         DET      0.989     0.988     0.988     30034
        CONJ      0.990     0.996     0.993      8689
        NOUN      0.959     0.974     0.967     51226
        PRON      0.992     0.985     0.989     20297
         PRT      0.948     0.902     0.924     10020
         NUM      0.930     0.986     0.957      1753

    accuracy                          0.972    270503
   macro avg      0.945     0.907     0.917    270503
weighted avg      0.972     0.972     0.972    270503



In [ ]:
## calling pos_tagger to use my model
def pos_tagger(my_sentence):
    if type(my_sentence) == str:
        doc = nlp(my_sentence)
        sentence = [word.text for word in doc]
        my_Features = token2features(sentence)
    prediction = crf.predict([my_Features])
    return prediction

## creating Input feild

In [ ]:
my_text = input('Isert an example: ')
print("\n",pos_tagger(my_text)[0])

Isert an example: I am a student

 ['PRON', 'VERB', 'DET', 'NOUN']
